In [0]:
"""
КНН с использованием евклидова и косинусного расстояния
"""

import csv
import random
import math
import operator 
from scipy.spatial.distance import cosine 

In [0]:
def load_dataset(filename, split):
    """
     имя файла: имя файла, который нужно открыть
     разделение параметров: отношение данных к обучающему набору
     возврат: учебные и тестовые списки
    """
    
   
    training_set = []
    test_set = []
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset) - 1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                training_set.append(dataset[x])
            else:
                test_set.append(dataset[x])

    return training_set, test_set

In [0]:
def euclidean_distance(instance1, instance2, length):
    """
Рассчитывает евклидово расстояние между двумя точками
     параметр instance1: точка данных 1
     параметр instance2: точка данных 2
     длина параметра: сколько измерений нужно оценить
     возврат: расстояние
    """
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

In [0]:
def cosine_distance(instance1, instance2):
    " Предполагем, что последний элемент каждого экземпляра является меткой или классом "
    p1 = instance1[:-1]
    p2 = instance2[:-1]
    return cosine(p1, p2)

In [0]:
def get_neighbors(training_set, test_instance, k, cosine_d=False):
    """

     Даем контрольному экземпляру вернуть k в ближайший набор

    param training_set list of training data points
    param test_instance single point in the test set
    param k number of neighbors to use
    return list of k closest data points

    """
    distances = []
    length = len(test_instance) - 1
    for x in range(len(training_set)):
        if cosine_d:
            dist = cosine_distance(test_instance, training_set[x])
            distances.append((training_set[x], dist))
        else:
            dist = euclidean_distance(test_instance, training_set[x], length)
            distances.append((training_set[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


In [0]:
def get_response(neighbors):
    """
    Последним соседним атрибутом является Ex: [1, 1, 1, a]: соседние параметры: список ближайших точек к точке
    """
    class_votes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in class_votes:
            class_votes[response] += 1
        else:
            class_votes[response] = 1
    sorted_votes = sorted(class_votes.items(), key=operator.itemgetter(1), reverse=True)
    return sorted_votes[0][0]

In [0]:
def get_accuracy(test_set, predictions):
    """

    параметр test_set: набор данных для тестирования
    предсказания параметров: метки, основанные на kNN
    возврат: правильный процент
    """
    correct = 0
    for x in range(len(test_set)):
        if test_set[x][-1] == predictions[x]:
            correct += 1
    return (correct / float(len(test_set))) * 100.0


In [ ]:
def main(cosine_d=False):
    # prepare data
    split = 0.67
    training_set, test_set = load_dataset(filename, split)
    print('Train set: ' + repr(len(training_set)))
    print('Test set: ' + repr(len(test_set)))
    # generate predictions
    predictions = []
    k = 3

    if cosine_d:
        for x in range(len(test_set)):
            neighbors = get_neighbors(training_set, test_set[x], k)
            result = get_response(neighbors)
            predictions.append(result)
            print('> predicted=' + repr(result) + ', actual=' + repr(test_set[x][-1]))
        accuracy = get_accuracy(test_set, predictions)
        print('Accuracy: ' + repr(accuracy) + '%')

    else:
        for x in range(len(test_set)):
            neighbors = get_neighbors(training_set, test_set[x], k)
            result = get_response(neighbors)
            predictions.append(result)
            print('> predicted=' + repr(result) + ', actual=' + repr(test_set[x][-1]))
        accuracy = get_accuracy(test_set, predictions)
        print('Accuracy: ' + repr(accuracy) + '%')


main()
main(cosine_d=True)
